In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
data = pd.read_csv('/content/Combined_rssi_corrected_proper_sinr_row_fixed (1).csv',delimiter=',')

df2=data.loc[data['node_type']==1]
del df2['node_type']

#data=data.loc[data[0]==0]
#data.replace([np.inf,-np.inf],np.nan)
#data.dropna(inplace=True)
#data = data.drop("node_code", axis=1)
#pd.to_numeric(data.node_type)
#data['node_type']=pd.get_dummies(data['node_type'])
X = df2.iloc[:,:-1].values
y = df2.iloc[:,7].values


In [ ]:
data

,node_type,x(m),y(m),primary_channel,min_channel_allowed,max_channel_allowed,RSSI,SINR,throughput
0,0,10.0000,10.0000,4,4,5,20.00,10000.00,111.77
1,1,0.0713,10.8079,4,4,5,-65.37,29.36,5.79
2,1,1.9627,4.1427,4,4,5,-65.35,29.48,6.11
3,1,13.7849,16.7538,4,4,5,-61.41,31.97,6.91
4,1,6.7112,1.7487,4,4,5,-63.52,31.26,9.99
...,...,...,...,...,...,...,...,...,...
78073,1,24.1359,33.5025,0,0,3,-60.91,20.98,3.99
78074,1,31.7151,27.9136,0,0,3,-63.54,18.54,9.60
78075,1,25.6285,29.0904,0,0,3,-63.65,16.51,2.92
78076,1,31.1102,32.1255,0,0,3,-52.59,31.73,14.05


In [ ]:
df2

,x(m),y(m),primary_channel,min_channel_allowed,max_channel_allowed,RSSI,SINR,throughput
1,0.0713,10.8079,4,4,5,-65.37,29.36,5.79
2,1.9627,4.1427,4,4,5,-65.35,29.48,6.11
3,13.7849,16.7538,4,4,5,-61.41,31.97,6.91
4,6.7112,1.7487,4,4,5,-63.52,31.26,9.99
5,13.1934,2.3628,4,4,5,-62.42,32.05,9.44
...,...,...,...,...,...,...,...,...
78073,24.1359,33.5025,0,0,3,-60.91,20.98,3.99
78074,31.7151,27.9136,0,0,3,-63.54,18.54,9.60
78075,25.6285,29.0904,0,0,3,-63.65,16.51,2.92
78076,31.1102,32.1255,0,0,3,-52.59,31.73,14.05


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Model

In [ ]:
input_layer = Input(shape=(X.shape[1],))
dense_layer_1 = Dense(1024, activation='relu')(input_layer)
dense_layer_2 = Dense(1024, activation='relu')(dense_layer_1)
dense_layer_3 = Dense(1024, activation='relu')(dense_layer_2)
dense_layer_4 = Dense(1024, activation='relu')(dense_layer_3)
dense_layer_5 = Dense(1024, activation='relu')(dense_layer_4)
dense_layer_6 = Dense(1024, activation='relu')(dense_layer_5)
dense_layer_7 = Dense(512, activation='relu')(dense_layer_6)

output = Dense(1)(dense_layer_7)

model = Model(inputs=input_layer, outputs=output)

model.compile(loss="mean_squared_error" , optimizer="adam", metrics=["mean_squared_error","mean_absolute_error"])

In [ ]:
history = model.fit(X, y, batch_size=250, epochs=1000)

Epoch 1/1000
289/289 [==============================] - 2s 5ms/step - loss: 42.3865 - mean_squared_error: 42.3865 - mean_absolute_error: 4.3649
Epoch 2/1000
289/289 [==============================] - 1s 5ms/step - loss: 36.2545 - mean_squared_error: 36.2545 - mean_absolute_error: 4.0599
Epoch 3/1000
289/289 [==============================] - 1s 5ms/step - loss: 35.8262 - mean_squared_error: 35.8262 - mean_absolute_error: 4.0305
Epoch 4/1000
289/289 [==============================] - 1s 5ms/step - loss: 35.4887 - mean_squared_error: 35.4887 - mean_absolute_error: 4.0137
Epoch 5/1000
289/289 [==============================] - 1s 5ms/step - loss: 35.1708 - mean_squared_error: 35.1708 - mean_absolute_error: 3.9890
Epoch 6/1000
289/289 [==============================] - 1s 5ms/step - loss: 34.1394 - mean_squared_error: 34.1394 - mean_absolute_error: 3.9239
Epoch 7/1000
289/289 [==============================] - 1s 5ms/step - loss: 34.0575 - mean_squared_error: 34.0575 - mean_absolute_error:

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import r2_score
from math import sqrt
pred_train = model.predict(X)

print(np.sqrt(mean_squared_error(y,pred_train)))#RMSE
print(mean_absolute_error(y,pred_train))#MAE
print(r2_score(y,pred_train))#R2_score
print(1-(1-r2_score(y, pred_train))*((len(X)-1)/(len(X)-len(X[0])-1)))#adjusted R2

1.323148154925778
0.6970560480631457
0.9642256911276045
0.9642222164479582


In [ ]:
pred_train

array([[ 5.350766 ],
       [ 6.293396 ],
       [ 7.202733 ],
       ...,
       [ 2.7573698],
       [14.158258 ],
       [ 7.46682  ]], dtype=float32)

In [ ]:
from keras.models import load_model

#model.save('Final_ANN_ITU_predictor_500_epochs.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
#model = load_model('Final_ANN_ITU_predictor.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/AI-ML"

Mounted at /content/gdrive
All_combined_proper_clean_final.zip  Final_ANN_ITU_predictor_500_epochs.h5
ANN_predictor_new.py		     Untitled0.ipynb
ANN_PRED.ipynb


In [ ]:
#model = load_model('/content/drive/My Drive/ITU/Final/Final_ANN_ITU_predictor.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip '/content/drive/My Drive/AI-ML/All_combined_proper_clean_final.zip'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/My Drive/AI-ML/All_combined_proper_clean_final.zip
   creating: All_combined_proper_clean_final/
  inflating: All_combined_proper_clean_final/input_nodes_test_sce1_deployment_000new.csv  
  inflating: All_combined_proper_clean_final/input_nodes_test_sce1_deployment_001new.csv  
  inflating: All_combined_proper_clean_final/input_nodes_test_sce1_deployment_002new.csv  
  inflating: All_combined_proper_clean_final/input_nodes_test_sce1_deployment_003new.csv  
  inflating: All_combined_proper_clean_final/input_nodes_test_sce1_deployment_004new.csv  
  inflating: All_combined_proper_clean_final/input_nodes_test_sce1_deployment_005new.csv  
  inflating: All_combined_proper_clean_final/input_nodes_test_sce1_deployment_006new.csv  
  inflating: All_combined_proper_clean_final/input_nodes_test_sce1_deployment_007new.csv  
  inflating: All_comb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
import re
import glob
'''
df=pd.read_csv('Combined_rssi_corrected_proper_sinr_row_fixed.csv')
df2=df.loc[df['node_type']==1]
del df2['node_type']
X= df2.iloc[:, :-1].values
y = df2.iloc[:, 7].values
sc = StandardScaler()
X = sc.fit_transform(X)
model = RandomForestmodelessor(max_depth=10, random_state=0)
model.fit(X, y) # Training done here
'''
#filepath='Final_ANN_ITU_predictor_500_epochs.h5'
#model=load_model(filepath)

def predictor(filename):
  file=re.sub('.csv', '', filename)
  data=pd.read_csv(filename)# The file needed to be predicted
  AP_list_index=(data.loc[data['node_type']==0].index.values)
  AP_list=AP_list_index.tolist()
  data_2= data.loc[data['node_type']==1]
  index_list=data_2.index.values.tolist()
  data_3= data_2
  data_pred=data_2
  del data_pred['node_type']
  X_test=data_pred.values
  X_test=sc.fit_transform(X_test)
  pred_model=model.predict(X_test)
  pred_model_list=pred_model.tolist()
  flat_list=[]
  for sublist in pred_model_list:
    for item in sublist:
      flat_list.append(item)
  #print(pred_model_list)
  pred_model_list= list(map(abs,flat_list))
  predicted_dict={'Index':index_list,'Throughput':pred_model_list}
  predicted_output=pd.DataFrame(predicted_dict)
  predicted_output=predicted_output.set_index('Index')
  pred_changer=predicted_output
  for i in AP_list:
      line = pd.DataFrame({"Throughput": 1.3}, index=[i])
      pred_changer = pred_changer.append(line, ignore_index=False)
      pred_changer= pred_changer.sort_index()
  line = pd.DataFrame({"Throughput": 1.3}, index=[len(pred_changer.index)+1])
  pred_changer = pred_changer.append(line, ignore_index=True)
  AP_list_index_new=(pred_changer.loc[pred_changer['Throughput']==1.3].index.values)
  AP_list_new=AP_list_index_new.tolist()
  rng_list=[]
  for i in range(len(AP_list_new)):
      rng_list.append([AP_list_new[i-1],AP_list_new[i]])
  del rng_list[0]
  rng_values_list=[]
  for i in rng_list:
      rng_values_list.append(list(range(i[0],i[1],1)))
  for i in rng_values_list:
      del i[0]
  respective_throughput_values=[]
  for i in rng_values_list:
      #print(type(pred_changer['Throughput'].iloc[i].values))
      respective_throughput_values.append(pred_changer['Throughput'].iloc[i].values)
  AP_throughput_value=[]
  for i in respective_throughput_values:
      AP_throughput_value.append(np.sum(i))
  final_pred_df=pred_changer.loc[pred_changer['Throughput']!=1.3]
  for i,j in zip(AP_list,AP_throughput_value):
      line = pd.DataFrame({"Throughput":j}, index=[i])
      final_pred_df = final_pred_df.append(line, ignore_index=False)
  final_pred_df= final_pred_df.sort_index()
  final_pred_df_transpose=final_pred_df.T
  final_pred_df_transpose.to_csv(file+'_ANN_throughput.csv',index=False,header=False)#Change the name of the file here to suit the output

pathname='/content/All_combined_proper_clean_final/'

for f in glob.glob(pathname+'*new.csv'):
	predictor(f)

In [ ]:
!zip -r '/content/ANN_1000_online_throughput+oct_9.zip' '/content/All_combined_proper_clean_final'

  adding: content/All_combined_proper_clean_final/ (stored 0%)
  adding: content/All_combined_proper_clean_final/input_nodes_test_sce3_deployment_006new_ANN_throughput.csv (deflated 49%)
  adding: content/All_combined_proper_clean_final/input_nodes_test_sce2_deployment_009new_ANN_throughput.csv (deflated 48%)
  adding: content/All_combined_proper_clean_final/input_nodes_test_sce2_deployment_026new_ANN_throughput.csv (deflated 47%)
  adding: content/All_combined_proper_clean_final/input_nodes_test_sce1_deployment_004new_ANN_throughput.csv (deflated 45%)
  adding: content/All_combined_proper_clean_final/input_nodes_test_sce2_deployment_045new_ANN_throughput.csv (deflated 48%)
  adding: content/All_combined_proper_clean_final/input_nodes_test_sce2_deployment_015new.csv (deflated 56%)
  adding: content/All_combined_proper_clean_final/input_nodes_test_sce3_deployment_004new.csv (deflated 59%)
  adding: content/All_combined_proper_clean_final/input_nodes_test_sce1_deployment_049new.csv (defl